In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [2]:
# Read the CSV file
df = pd.read_csv("dataset.csv")

# Define the model
model = nn.Sequential(
    nn.Linear(3, 16),
    nn.LeakyReLU(),
    nn.BatchNorm1d(16),
    nn.Dropout(0.1),
    nn.Linear(16, 20),
    nn.Tanh(),
    nn.Dropout(0.2),
    nn.Linear(20, 12),
    nn.LeakyReLU(),
    nn.BatchNorm1d(12),
    nn.Dropout(0.3),
    nn.Linear(12, 3),
    nn.Sigmoid(),
    nn.BatchNorm1d(3),
    nn.Linear(3, 1)
).cuda()


In [3]:
# Define the loss function
criterion = nn.BCEWithLogitsLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Preprocess the data
le = LabelEncoder()
df['class'] = le.fit_transform(df['class'])
X = df.drop('class', axis=1).values
y = df['class'].values

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float).cuda()
y_train = torch.tensor(y_train, dtype=torch.float).cuda()
X_test = torch.tensor(X_test, dtype=torch.float).cuda()
y_test = torch.tensor(y_test, dtype=torch.float).cuda()

# Create data loaders
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=128, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=1024, shuffle=True)

In [ ]:
for epoch in tqdm(range(100)):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        targets = targets.view(-1, 1)  # Add this line
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

In [4]:
# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, targets in test_loader:
        outputs = model(inputs)
        targets = targets.view(-1, 1)  # Add this line
        predicted = torch.round(torch.sigmoid(outputs))
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

print('Accuracy on test data: %d %%' % (100 * correct / total))

Accuracy on test data: 77 %
